# Prompt Parallelization Workflow
In this notebook, we will build a content moderation system specifically designed for media and entertainment companies using the Prompt Parallelization pattern with Langgraph Agent. This pattern allows us to process media content through multiple specialized LLM prompts in parallel and then aggregate the results.

At a high level, we'll be creating a media content moderation system that:

1. Takes content as input (scripts, user comments, social posts, etc.)
2. Analyzes it from multiple perspectives in parallel:
  - Explicit language detection and age-appropriate terminology
  - Inappropriate themes evaluation and cultural sensitivity
  - Misleading content identification and copyright concerns
  - Sentiment analysis and brand alignment assessment
3. Aggregates the results into a comprehensive moderation report tailored for media platforms


The following diagram depicts the workflow for a parallelization workflow implemented in Langgraph:

<img src="../../imgs/strands-parallel-llm-architecture.png" width="800"> 


### Required Imports
Import all necessary libraries and modules for the content moderation workflow:

In [ ]:
from langchain_aws import ChatBedrockConverse
import boto3
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

## Logging Configuration

Set up logging to track the execution of our content moderation workflow. This helps with debugging and monitoring the performance of each parallel analysis agent.


In [ ]:
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

## AWS Bedrock Client Setup

Initialize the AWS Bedrock client for accessing foundation models. The client will be used by all analysis agents to make LLM calls for content moderation tasks.


In [ ]:
session = boto3.session.Session()
region = session.region_name
bedrock_client = boto3.client('bedrock-runtime', region)

## Graph State Definition

Define the state structure that will be shared across all nodes in the LangGraph workflow. This state contains:
- **content**: The input content to be analyzed
- **explicitLanguageAnalysis**: Results from explicit language detection agent
- **inappropriateThemeAnalysis**: Results from theme analysis agent  
- **misleadingContentAnalysis**: Results from misleading content agent
- **sentimentAnalysis**: Results from sentiment analysis agent
- **final_result**: Aggregated moderation report


In [ ]:
# Graph state
class State(TypedDict):
    content: str
    explicitLanguageAnalysis: str
    inappropriateThemeAnalysis: str
    misleadingContentAnalysis: str
    sentimentAnalysis: str
    final_result: str


## Explicit Language Analysis Agent

This agent specializes in detecting explicit language and determining age-appropriate content ratings. It analyzes:
- Profanity and explicit language
- Age-appropriate terminology  
- Content suitability for different audience ratings (G, PG, PG-13, R, etc.)
- Dialogue requiring parental guidance

The agent returns a structured JSON response with assessment, specific instances found, age rating recommendation, and revision suggestions.


In [ ]:
def explicitLanguageAnalysisAgent(state: State):
    system_prompt = """You are a content moderator for a major media and entertainment company specializing in identifying explicit language and age-appropriate content.

# Analyze the following content for:

1. Explicit language and profanity
2. Age-appropriate terminology
3. Content suitable for different audience age ratings (G, PG, PG-13, R, etc.)
4. Dialogue or descriptions that may require parental guidance

## Provide a detailed report with:

1. Overall assessment (Family-Friendly/Teen-Appropriate/Mature/Adult-Only)
2. Specific instances of explicit language or age-inappropriate content found (if any)
3. Age rating recommendation (G, PG, PG-13, R, NC-17, etc.)
4. Recommendations for content revision to meet specific audience age targets

Format your response as JSON:

   {
      "assessment": "Family-Friendly/Teen-Appropriate/Mature/Adult-Only",
      "instances": ["example1", "example2"],
      "age_rating": "PG-13",
      "revision_recommendations": "Your recommendations here"
   }
"""
    """First LLM call to generate initial joke"""
    model_id = "us.amazon.nova-micro-v1:0"
    llm = ChatBedrockConverse(
        model=model_id,  # or another Claude model
        temperature=0.3,
        max_tokens=None,
        client=bedrock_client,
    )
    
    messages = [ ("system", system_prompt), ("human", f"Content to analyze: {state['content']}")]
    msg = llm.invoke(messages)
    return {"explicitLanguageAnalysis": msg.content}

## Inappropriate Theme Analysis Agent

This agent identifies inappropriate themes and cultural sensitivity issues in content. It evaluates:
- Violence, discrimination, or harmful ideologies
- Cultural sensitivity concerns for global audiences
- Offensive stereotypes or representations
- Content problematic for certain regions or markets

The agent provides assessment, identified themes, sensitive regions, and recommendations for improving cultural sensitivity.


In [ ]:
def inappropriateThemeAnalysisAgent(state: State):
   system_prompt = """You are a content moderator for a major media and entertainment company specializing in identifying inappropriate themes and cultural sensitivity issues.

# Analyze the following content for:

1. Inappropriate themes such as violence, discrimination, or harmful ideologies
2. Cultural sensitivity concerns across global audiences
3. Potentially offensive stereotypes or representations
4. Content that may be inappropriate for certain regions or markets


## Provide a detailed report with:

1. Overall assessment (Universally Acceptable/Regionally Sensitive/Potentially Offensive)
2. Specific inappropriate themes or cultural sensitivity issues identified (if any)
3. Regions or demographics that may find the content problematic
4. Recommendations for content revision to improve cultural sensitivity

# Format your response as JSON:

   {
      "assessment": "Universally Acceptable/Regionally Sensitive/Potentially Offensive",
      "themes": ["theme1", "theme2"],
      "sensitive_regions": ["region1", "region2"],
      "revision_recommendations": "Your recommendations here"
   }
"""
   model_id = "us.amazon.nova-micro-v1:0"
   llm = ChatBedrockConverse(
      model=model_id,  # or another Claude model
      temperature=0.3,
      max_tokens=None,
      client=bedrock_client,
   )
   messages = [ ("system", system_prompt), ("human", f"Content to analyze: {state['content']}")]
   msg = llm.invoke(messages)
   return {"inappropriateThemeAnalysis": msg.content}

## Misleading Content Analysis Agent

This agent focuses on factual accuracy and potential copyright issues. It examines:
- Misleading claims or factual inaccuracies
- Potential copyright or intellectual property concerns
- Unauthorized references to protected brands, characters, or properties
- Content requiring fact-checking or legal review

The agent returns assessment of factual soundness, identified inaccuracies, copyright concerns, and revision recommendations.


In [ ]:
def misleadingContentAnalysisAgent(state: State):
   system_prompt = """You are a content moderator for a major media and entertainment company specializing in identifying misleading content, factual inaccuracies, and potential copyright issues.

# Analyze the following content for:

1. Misleading claims or factual inaccuracies
2. Potential copyright or intellectual property concerns
3. Unauthorized references to protected brands, characters, or properties
4. Content that may require fact-checking or legal review

## Provide a detailed report with:

1. Overall assessment (Factually Sound/Needs Verification/Potentially Infringing)
2. Specific misleading claims or factual inaccuracies identified (if any)
3. Potential copyright or IP concerns
4. Recommendations for content revision to address factual or legal issues

Format your response as JSON:

   {
      "assessment": "Factually Sound/Needs Verification/Potentially Infringing",
      "inaccuracies": ["inaccuracy1", "inaccuracy2"],
      "copyright_concerns": ["concern1", "concern2"],
      "revision_recommendations": "Your recommendations here"
   }
"""
   model_id = "us.amazon.nova-lite-v1:0"
   llm = ChatBedrockConverse(
         model=model_id,  # or another Claude model
         temperature=0.3,
         max_tokens=None,
         client=bedrock_client,
      )
   messages = [ ("system", system_prompt), ("human", f"Content to analyze: {state['content']}")]
   msg = llm.invoke(messages)
   return {"misleadingContentAnalysis": msg.content}


## Sentiment Analysis Agent

This agent analyzes emotional tone and brand alignment of content. It evaluates:
- Overall emotional tone and sentiment
- Alignment with media brand values (positivity, inclusivity, creativity)
- Potential impact on audience perception and brand reputation
- Emotional resonance with target demographics

The agent provides sentiment classification, brand alignment assessment, emotional impact analysis, and recommendations for brand consistency.


In [ ]:
def sentimentAnalysisAgent(state: State):
    system_prompt = """You are a content moderator for a major media and entertainment company specializing in sentiment analysis and brand alignment.

# Analyze the following content for:

1. Overall emotional tone and sentiment
2. Alignment with typical media brand values (positivity, inclusivity, creativity, etc.)
3. Potential impact on audience perception and brand reputation
4. Emotional resonance with target demographics

# Provide a detailed report with:
1. Overall sentiment (Positive/Neutral/Negative)
2. Brand alignment assessment (Aligned/Partially Aligned/Misaligned)
3. Potential emotional impact on viewers/readers
4. Recommendations for content revision to improve brand alignment

Format your response as JSON:

{
    "sentiment": "Positive/Neutral/Negative",
    "brand_alignment": "Aligned/Partially Aligned/Misaligned",
    "emotional_impact": "Your assessment here",
    "revision_recommendations": "Your recommendations here"
}
"""
    model_id = "us.amazon.nova-lite-v1:0"
    llm = ChatBedrockConverse(
            model=model_id,  # or another Claude model
            temperature=0.3,
            max_tokens=None,
            client=bedrock_client,
        )
    messages = [ ("system", system_prompt), ("human", f"Content to analyze: {state['content']}")]
    msg = llm.invoke(messages)
    return {"sentimentAnalysis": msg.content}


## Aggregator Agent

This agent synthesizes all parallel analysis results into a comprehensive moderation report. It combines insights from all four analysis agents to provide:
- Overall moderation decision (Approve/Revise/Reject)
- Summary of key concerns from all analyses
- Platform-specific recommendations for different distribution channels
- Content tags and warnings
- Detailed justification for the final decision

In [ ]:
def aggregator(state: State):
    system_prompt = """You are a senior content moderator for a major media and entertainment company responsible for creating comprehensive moderation reports with platform-specific recommendations.

   You are given the moderation analysis reports as described below:
   
   - Explicit Language & Age-Appropriate Analysis
   - Inappropriate Themes & Cultural Sensitivity Analysis
   - Misleading Content & Copyright Analysis
   - Sentiment & Brand Alignment Analysis


   Based on these reports, provide:
   1. Overall moderation decision (Approve/Revise/Reject)
   2. Summary of key concerns from all analyses
   3. Platform-specific recommendations for:
      - Streaming platforms (Netflix, Disney+, Prime Video, etc.)
      - Broadcast television
      - Social media platforms
      - International distribution
   4. Specific content tags or warnings recommended
   5. Justification for your decision

   Format your response as a well-structured report with clear sections for each platform type.
"""
    model_id = "us.amazon.nova-pro-v1:0"
    llm = ChatBedrockConverse(
            model=model_id,  # or another Claude model
            temperature=0.3,
            max_tokens=None,
            client=bedrock_client,
        )
    
    explicitLanguageAnalysis = state["explicitLanguageAnalysis"]
    inappropriateThemeAnalysis = state["inappropriateThemeAnalysis"]
    misleadingContentAnalysis = state["misleadingContentAnalysis"]
    sentimentAnalysis = state["sentimentAnalysis"]

    mediaPlatformRecommendations_prompt = f"""## Explicit Language & Age-Appropriate Analysis Report:

{explicitLanguageAnalysis}

## Inappropriate Themes & Cultural Sensitivity Analysis Report:

{inappropriateThemeAnalysis}

## Misleading Content & Copyright Analysis Report:

{misleadingContentAnalysis}


## Sentiment & Brand Alignment Analysis Report:

{sentimentAnalysis}

"""

    messages = [ ("system", system_prompt), ("human", f"{mediaPlatformRecommendations_prompt}")]
    msg = llm.invoke(messages)
    return {"final_result": msg.content}

## Workflow Graph Construction

Build the LangGraph workflow that orchestrates parallel execution of all analysis agents:

1. **Add Nodes**: Register all analysis agents and the aggregator as nodes
2. **Parallel Edges**: Connect START to all four analysis agents simultaneously  
3. **Convergence**: All analysis agents feed their results to the aggregator
4. **Completion**: Aggregator produces final result and connects to END

The graph visualization shows the parallel execution pattern where multiple analysis agents run concurrently before converging at the aggregator.


In [ ]:
# Build workflow
parallel_builder = StateGraph(State)

# Add nodes
parallel_builder.add_node("explicitLanguageAnalysisAgent", explicitLanguageAnalysisAgent)
parallel_builder.add_node("inappropriateThemeAnalysisAgent", inappropriateThemeAnalysisAgent)
parallel_builder.add_node("misleadingContentAnalysisAgent", misleadingContentAnalysisAgent)
parallel_builder.add_node("sentimentAnalysisAgent", sentimentAnalysisAgent)
parallel_builder.add_node("aggregator", aggregator)

# Add edges to connect nodes
parallel_builder.add_edge(START, "explicitLanguageAnalysisAgent")
parallel_builder.add_edge(START, "inappropriateThemeAnalysisAgent")
parallel_builder.add_edge(START, "misleadingContentAnalysisAgent")
parallel_builder.add_edge(START, "sentimentAnalysisAgent")
parallel_builder.add_edge("explicitLanguageAnalysisAgent", "aggregator")
parallel_builder.add_edge("inappropriateThemeAnalysisAgent", "aggregator")
parallel_builder.add_edge("misleadingContentAnalysisAgent", "aggregator")
parallel_builder.add_edge("sentimentAnalysisAgent", "aggregator")
parallel_builder.add_edge("aggregator", END)
parallel_workflow = parallel_builder.compile()

# Show workflow
display(Image(parallel_workflow.get_graph().draw_mermaid_png()))

## Sample Content for Analysis

This is a sample crime drama script excerpt that will be used to test the content moderation workflow. The script contains elements that will trigger different analysis agents:
- Explicit language and mature themes
- Violence and crime-related content  
- Potentially sensitive material requiring age ratings
- Content that may need moderation for different platforms

This realistic example demonstrates how the parallel analysis system handles complex, multi-faceted content.


In [ ]:
script = """FADE IN:

   EXT. CITY STREET - NIGHT

   Rain pours down as DETECTIVE MORGAN (45, hardened) stands over a body covered by a sheet. Lightning flashes, illuminating the grim scene.

   DETECTIVE MORGAN
   (lighting a cigarette)
   Third one this month. Same damn signature.

   OFFICER JENKINS (30s) approaches, looking queasy.

   OFFICER JENKINS
   The mayor's going to have a fit. The press is already calling this guy "The Midnight Butcher."

   DETECTIVE MORGAN
   I don't give a damn what they call him. I'm going to catch this son of a bitch before he cuts up another girl.

   Morgan kneels down and pulls back the sheet, revealing a young woman's face, pale and lifeless.

   DETECTIVE MORGAN
   (whispering)
   I promise you justice, sweetheart. Whatever it takes.

   He stands, tosses his cigarette into a puddle.

   DETECTIVE MORGAN
   Jenkins, get me everything on the victim. And I mean everything. Where she worked, who she dated, what goddamn toothpaste she used.

   OFFICER JENKINS
   But sir, it's almost midnight and—

   DETECTIVE MORGAN
   (interrupting)
   Evil doesn't sleep, why should we?

   Morgan walks away as lightning flashes again, his silhouette momentarily frozen against the night sky.

   FADE OUT.
   """

## Execute the Workflow

Run the complete content moderation workflow on the sample script. The parallel execution will:

1. **Simultaneously analyze** the content across four dimensions (explicit language, themes, misleading content, sentiment)
2. **Aggregate results** into a comprehensive moderation report
3. **Provide platform-specific recommendations** for content distribution

The output shows the final moderation decision and detailed analysis from all agents, demonstrating the power of parallel processing for comprehensive content review.


In [ ]:
# Invoke
state = parallel_workflow.invoke({"content": script})
print(state["final_result"])

## Summary
The notebook demonstrates how to use Langgraph framework to implement the **Prompt Parallelization** pattern, a powerful approach for processing content through multiple specialized LLM prompts simultaneously. Unlike sequential processing, this pattern enables parallel execution to dramatically improve efficiency and provide comprehensive analysis.

## Key Takeaways

1. **Parallel > Sequential**: Dramatic performance improvements through concurrent processing
2. **Specialization Works**: Domain-specific agents provide higher quality analysis than general-purpose approaches
3. **Scalable Architecture**: Pattern easily extends to additional analysis dimensions
